### Imports

In [22]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import json

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import torchvision
from tqdm.notebook import tqdm






### Constants

In [23]:
DIR = 'datasets\\trials'
BATCH_SIZE = 1
MAX_FRAMES = 2
VIT_OUT_DIM = 1000

In [24]:
np.array([1,2,3]).shape

(3,)

# Datasets

In [25]:
frames = []
infos = []

for trial_fp in os.listdir(DIR):
    trial_fp = os.path.join(DIR, trial_fp)
    imgs = []
    for fp in os.listdir(trial_fp):
        fp = os.path.join(trial_fp, fp)
        if fp[-4:] == '.png':
            imgs.append(np.rollaxis(np.array(Image.open(fp), dtype=np.float32),2,0))
        else:
            infos.append(json.load(open(fp)))
    frames.append(np.array(imgs))

In [26]:
instructions = [x['instruction'] for x in infos]
target_actions = [x['answers'] for x in infos]

## Instruction Dataset

In [27]:
class InstructionsDataset(Dataset):
  """
    Pytorch Dataset class to load the Instructions Data

    Data members:
      instructions: list of instructions
      n_ins: number of instructions in the dataset

    Member functions:
      __init__: ctor
      __len__: returns n_ins
      __getitem__: returns an instruction
  """

  def __init__(self, x):

    self.instructions = x

    self.n_ins = len(self.instructions)

    return

  def __len__(self):
    """
      Returns number of instructions in the Dataset
    """

    return self.n_ins

  def __getitem__(self, idx):
    """
      Given an index return a instruction at that index
    """

    return self.instructions[idx]

In [28]:
class InstructionsCollator(object):
  """
    Data Collator used for GPT2 in a classificaiton tasks

    Args:
      use_tokenizer :
        Transformer type tokenizer used to process raw text into numbers.

    Data members:
      use_tokenizer: Tokenizer to be used inside the class.

    Member functions:
      __init__: ctor
      __call__: tokenize input

    """

  def __init__(self, use_tokenizer):

    self.use_tokenizer = use_tokenizer

    return

  def __call__(self, instructions):
    """
        Tokenizes input
    """

    # Call tokenizer
    inputs = self.use_tokenizer(instructions, padding=True, truncation=True, return_tensors='pt')


    return inputs


In [29]:
lm_encoder = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Create data collator to encode text and labels into numbers.
InstructionsCollator = InstructionsCollator(use_tokenizer=tokenizer)

# Create pytorch dataset for instructions
ins_train_dataset = InstructionsDataset(instructions)

# Move pytorch dataset into dataloader 
ins_train_dataloader = DataLoader(ins_train_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=InstructionsCollator)


## Frames Dataset

In [30]:
class FramesDataset(Dataset):
  """
    Pytorch Dataset class to load the Frame Data

    Data members:
      frames``ist of frames
      n_imgs: number of iamges in the dataset

    Member functions:
      __init__: ctor
      __len__: returns n_imgs
      __getitem__: returns an frame
  """

  def __init__(self, x):

    self.frames = x

    self.n_imgs = len(self.frames)

    return

  def __len__(self):
    """
      Returns number of frames in the Dataset
    """

    return self.n_imgs

  def __getitem__(self, idx):
    """
      Given an index return a frame
    """

    return torch.tensor(self.frames[idx])

In [31]:
vit_encoder = torchvision.models.vit_b_16(weights=torchvision.models.ViT_B_16_Weights.DEFAULT)

# Create pytorch dataset for instructions
frames_train_dataset = FramesDataset(frames)

# Move pytorch dataset into dataloader 
frames_train_dataloader = DataLoader(frames_train_dataset, batch_size=BATCH_SIZE, shuffle=False)


# Language Encoder

### Language Embedder

In [32]:
def lm_embedder(batch, encoder):
    #Mean Pooling - Take attention mask into account for correct averaging
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


    # Compute token embeddings
    with torch.no_grad():
        lm_output = encoder(**batch)
    # Perform pooling
    sentence_embeddings = mean_pooling(lm_output, batch['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        
    return sentence_embeddings

# Image Encoder

### Position Embeddings

In [67]:
pos_emb = nn.Embedding(MAX_FRAMES,VIT_OUT_DIM)
pos_emb.weight = nn.init.xavier_uniform_(pos_emb.weight)

Parameter containing:
tensor([[ 1.2204,  0.9137,  1.7954,  ..., -1.2619,  0.2253,  1.2762],
        [ 1.6056,  0.9815, -0.1134,  ..., -0.4408, -2.1606, -1.4598]],
       requires_grad=True)
Parameter containing:
tensor([[-0.0560, -0.0742, -0.0485,  ...,  0.0471, -0.0291,  0.0240],
        [ 0.0679,  0.0007,  0.0299,  ...,  0.0196, -0.0595,  0.0222]],
       requires_grad=True)


In [ ]:
pos_emb

### Image Embedder

In [50]:
def img_embedder(frames, encoder):
    with torch.no_grad():
        vit_out = encoder(torch.tensor(frames))
    for i in range(len(frames)):
        vit_out[i] += pos_emb(torch.tensor([1]))



##

# Action Decoder

# Run Embeddings

In [35]:
lm_embeddings = []
img_embeddings = []

for i,f in zip(ins_train_dataloader,frames_train_dataloader):
   f = f[0]
   y = lm_embedder(i, 1, lm_encoder)
   lm_embeddings.append(lm_embedder(i, 1, lm_encoder))
   img_embeddings.append(img_embedder(vit_encoder(f)))

TypeError: lm_embedder() takes 2 positional arguments but 3 were given

# Evaluating